In [11]:
### FEITO POR LEONARDO FARIA LIMA 
### 12/07/2024
### REVISAO: 09/08/2024

In [12]:
import pandas as pd  # type: ignore
import time
import locale
import os

In [13]:
### MOSTRA OS ARQUIVOS DO DIRETÓRIO

# Define o caminho do diretório
diretorio = r"I:\Ger_Suporte a Cliente\Inteligencia de Mercado\005_Analytics\002_Bases\021_L2L\03_bases_e_template_lis"

# Lista os arquivos no diretório
arquivos = os.listdir(diretorio)

# Exiba os arquivos
print("Escolha o nome do arquivo: (.xlsx):")
print("\n")
for arquivo in arquivos:
    print(arquivo)


Escolha o nome do arquivo: (.xlsx):


base_precos_lis.csv
calculo_preco.xlsx
calculo_preco_Imunologia.xlsx
calculo_preco_OBMONOGENICA.xlsx
calculo_preco_paineis_neurais.xlsx
calculo_preco_RMCOMF_TCCOR.xlsx
calculo_preco_RMCOMF_TCCOR_bi.xlsx
envio_precos - Copia.xlsx
envio_precos.xlsx
envio_precos_DIV_OBMONO_2024-08-23_15h-54m.xlsx
envio_preco_backup
envio_preco_VIVO_RMCOMF_2024-08-28_15h-09m.xlsx
envio_preco_VIVO_RMCOMF_2024-08-28_16h-43m.xlsx
precos_base_calculo
template_lis.xlsx


In [14]:
### INPUT NOME BASE PRECOS 

nome_base_precos = input('Inserir o nome do arquivo referente a base de preços: ')

In [15]:
def trata_nome_arquivo (nome):
    nome = nome.replace("||","_")
    nome = nome.replace("|","_")
    nome = nome.upper()
    nome = nome.strip()
    return nome

In [16]:
#### INPUT DATAS E NOME ARQUIVO

locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

data_hoje = time.localtime() ### faz o unpacking da data 

ano = data_hoje.tm_year
mes = data_hoje.tm_mon
dia = data_hoje.tm_mday
dia_da_semana = data_hoje.tm_wday

if dia >= 1 and dia <= 9:
	dia = "0"+str(dia)
else:
	dia = str(dia)
    
if mes >= 1 and mes <=9:
	mes = "0"+str(mes)
else: 
	mes = str(mes) 

ano = str(ano)   
   
data_atual = dia+"-"+mes+"-"+ano
data_arquivo = time.strftime('%Y-%m-%d_%Hh-%Mm', data_hoje)

tempo_formatado = time.strftime("%A, %d de %B de %Y, %Hh:%Mm", data_hoje)
print(f">>>> Data e hora: {tempo_formatado}")
print('\n')   

>>>> Data e hora: quinta-feira, 29 de agosto de 2024, 15h:20m




In [17]:
### CUSTO DE PRODUÇÃO = CUSTO VARIÁVEL 
### VERIFICAR NA GPROD 

### 1. Premissas precificação do L2L 

* Custo Total (Custo de Produção) = Custo Variável Unitário + Custo Fixo Unitário. 
* Custo de Captação L2L = 10% do Custo Total. O custo de captação é composto por despesas com insumos de transporte + logística. 
* Custo Total L2L = Custo Total + Custo de Captação L2L, ou Custo Total L2L =  
* Margem preço mínimo: 15%. 
* Preço labor3: Custo Total / (1-imposto sobre faturamento)*(1-margem de lucro). 
* Preço Máximo: preço labor3 * 5.
* Menor preço absoluto (R$): se o novo preço sugerido for inferior a R$5,00, teremos que inserir no mínimo R$5,00; isso ocorre na maioria das vezes na família de analises clinicas e nas tabelas de funcionários.
* Preço (labor3) = Custo Total L2L / (1-imposto sobre faturamento) * (1-margem)



### 2. Tributação 

* Os impostos da venda com serviço de coleta e análise de exames são formados com base em PIS, COFINS, ISS de forma regionalizada. 
* Onde não houver dado usa-se 6,65% (média dos locais de maior volume).


In [18]:
# VARIÁVEIS 
tributo = 6.65/100
print('Imposto (%): ', tributo*100)

Imposto (%):  6.65


In [19]:
### CARREGA O ARQUIVO (BASE DE PREÇOS)

base_precos = r"I:\Ger_Suporte a Cliente\Inteligencia de Mercado\005_Analytics\002_Bases\021_L2L\03_bases_e_template_lis\{}".format(nome_base_precos)
df = pd.read_excel(base_precos)
display(df)

,exame,custo_total,gprod,mg_labor3,mg_minimo
0,VIVO||RMCOMF,251.25,0,34,15
1,VIVO||TCCOR,201.25,0,58,15


In [20]:
df['custo_capt_l2l'] = df['custo_total']*(10/100)  ### CÁCULO CUSTO DE CAPTAÇÃO L2L.  

In [21]:
#### IMPORTANTE: O CUSTO DE CAPTAÇAO L2L NÃO PODE EXCEDER R$ 50,00. 

df['custo_capt_l2l'] = df['custo_capt_l2l'].apply(lambda x: 50 if x > 50 else x)

display(df)

,exame,custo_total,gprod,mg_labor3,mg_minimo,custo_capt_l2l
0,VIVO||RMCOMF,251.25,0,34,15,25.125
1,VIVO||TCCOR,201.25,0,58,15,20.125


In [22]:
### CÁLCULOS DEMAIS VARIÁVEIS (labor3, mínimo, máximo e etc)

df["custo_l2l"] = df["custo_total"]+df["custo_capt_l2l"]
df["labor3"] = df["custo_l2l"]/((1-(df['mg_labor3']/100))*(1-tributo))

In [23]:
df["minimo"] = df["custo_l2l"]/((1-df["mg_minimo"]/100)*(1-tributo))
df["maximo"] = df['labor3']*5
df["mg_min%"] = (((df["minimo"]*(1-tributo))-df["custo_l2l"])/(df["minimo"]*(1-tributo)))*100
df["mg_labor3_custo%"] = (((df["labor3"]*(1-tributo))-df["custo_l2l"])/(df["labor3"]*(1-tributo)))*100
df["mg_labor3_min%"] = (((df["labor3"]*(1-tributo))-df["minimo"])/(df["labor3"]*(1-tributo)))*100
df["mg_min_r%"] = ((1-tributo)-(df["custo_l2l"]/df["minimo"]))*100
df["mg_labor3_r%"] = ((1-tributo)-(df["custo_l2l"]/df["labor3"]))*100

df = df.drop('mg_labor3', axis=1)

In [24]:
### ARREDONDAMENTO CASAS DECIMAIS 

df = df.round({'custo_total':2, 'custo_capt_l2l': 2, 'labor3': 2,'custo_l2l': 2,'minimo': 2, "maximo": 2, 'mg_min%': 1, "mg_labor3_custo%":1, "mg_labor3_min%": 1, 'mg_min_r%': 1, 'mg_labor3_r%': 1})

In [25]:
### CRIA O NOME DO ARQUIVO 

set_name = df
nome_arquivo = set_name.loc[0, "exame"]
nome_arquivo = trata_nome_arquivo(nome_arquivo)
print("Nome do arquivo a ser salvo", "preco_"+nome_arquivo)

Nome do arquivo a ser salvo preco_VIVO_RMCOMF


In [26]:
salva_preco = r"i:\Ger_Suporte a Cliente\Inteligencia de Mercado\005_Analytics\002_Bases\021_L2L\05_resultado_calculo_preco\preco_{}_{}.xlsx".format(nome_arquivo, data_arquivo)
df.to_excel(salva_preco, index=False)
display(df)

,exame,custo_total,gprod,mg_minimo,custo_capt_l2l,custo_l2l,labor3,minimo,maximo,mg_min%,mg_labor3_custo%,mg_labor3_min%,mg_min_r%,mg_labor3_r%
0,VIVO||RMCOMF,251.25,0,15,25.12,276.38,448.58,348.31,2242.90,15.0,34.0,16.8,14.0,31.7
1,VIVO||TCCOR,201.25,0,15,20.12,221.38,564.63,278.99,2823.16,15.0,58.0,47.1,14.0,54.1


### 3. Gera o arquivo necessário para criar as tabelas de carga do LIS e Citrix
* Arquivo usado pelo script "carga_de_preço_lis_labor3".

In [27]:
df_envio_preco = df
df_envio_preco = df_envio_preco[['exame', 'custo_total', 'custo_l2l', 'minimo',	'maximo',	'labor3',	'gprod']]
display(df_envio_preco)

,exame,custo_total,custo_l2l,minimo,maximo,labor3,gprod
0,VIVO||RMCOMF,251.25,276.38,348.31,2242.90,448.58,0
1,VIVO||TCCOR,201.25,221.38,278.99,2823.16,564.63,0


In [28]:
salva_envio_preco = r"I:\Ger_Suporte a Cliente\Inteligencia de Mercado\005_Analytics\002_Bases\021_L2L\03_bases_e_template_lis\envio_preco_{}_{}.xlsx".format(nome_arquivo, data_arquivo)
df_envio_preco.to_excel(salva_envio_preco, index=False)